# Comparison Evaluators(비교 평가자)
##### 비교 평가자는 두 개의 서로 다른 체인 또는 LLM 출력을 측정하는데 사용된다.
##### 두 언어 모델간의 A/B 테스트 또는 동일한 모델의 다른 버전 같은 비교 분석에 유리하다.
##### 일반적으로 PairwiseStringEvaluator 클래스에서 상속되고 두 문자열에 대한 비교 인터페이스를 제공한다.
##### 비교 평가자는 문자열 쌍에 대한 평가를 수행하고 평가 점수 및 기타 관련 세부 점수가 포함된 사전을 반환한다.
##### 사용자 정의 비교 평가기를 생성할려면 PairwiseStringEvaluator 클래스를 상속하고, _evaluate_string_pairs 메서드를 덮어쓴다. 혹시 비동기 평가가 필요한 경우 _aevaluate_string_pairs 메서드도 덮어쓴다.

##### 비교 평가기의 주요 방법과 속성은 아래와 같다.

*   evaluate_string_pairs : 출력 문자열 쌍을 평가한다. 사용자 정의 평가자를 생성할 땐 해당 함수를 덮어쓴다.
*   aevaluate_string_pairs : 출력 문자열 쌍을 비동기적으로 평가한다. 사용자 정의 평가자를 생성할 땐 해당 함수를 덮어쓴다.
*   requires_input : 해당 속성은 평가자에 입력 문자열이 필요한지 여부를 나타낸다.
*   requires_reference : 해당 속성은 평가자에 참조 레이블이 필요한지 여부를 나타낸다.



## 문자열쌍 비교
##### 주어진 입력에 대해 선호하는 예측을 선책하는 가장 간단하고 신뢰하는 자동화 방법은 pairwise_string 평가기를 사용하는 것이다.

```python
from langchain.evaluation import load_evaluator

evaluator = load_evaluator("labeled_pairwise_string")

evaluator.evaluate_string_pairs(
    prediction="there are three dogs",
    prediction_b="4",
    input="how many dogs are in the park?",
    reference="four",
)

{'reasoning': 'Both responses are relevant to the question asked, as they both provide a numerical answer to the question about the number of dogs in the park. However, Response A is incorrect according to the reference answer, which states that there are four dogs. Response B, on the other hand, is correct as it matches the reference answer. Neither response demonstrates depth of thought, as they both simply provide a numerical answer without any additional information or context. \n\nBased on these criteria, Response B is the better response.\n',
 'value': 'B',
 'score': 0}
```

##### 쌍 문자열 평가자는 evaluate_string_pairs 메서드를 사용하여 호출할 수 있다.

*   prediction : 첫 번째 모델, 체인 또는 프롬프트의 예측된 응답
*   prediction_b : 두 번째 모델, 체인 또는 프롬프의 예측된 응답
*   input : 입력 질문, 프롬프트 또는 기타 텍스트
*   reference : 참조 응답

##### 다음 값을 사용하여 사전을 반환한다.


*   reasoning : LLM 문자열 (사고 추론)
*   value : 'A' 또는 'B' (prediction or prediction_b)
*   score : value에서 매핑된 정수 0 또는 1






### reference 없는 평가자
##### reference를 사용할 수 없는 경우에도 선호하는 응답을 예측할 수 있다.
##### 평가 모델의 선호도를 무조건 반영하기 때문에 신뢰도가 낮다.

```python
from langchain.evaluation import load_evaluator

evaluator = load_evaluator("pairwise_string")

evaluator.evaluate_string_pairs(
    prediction="Addition is a mathematical operation.",
    prediction_b="Addition is a mathematical operation that adds two numbers to create a third number, the 'sum'.",
    input="What is addition?",
)

{'reasoning': 'Both responses are correct and relevant to the question. However, Response B is more helpful and insightful as it provides a more detailed explanation of what addition is. Response A is correct but lacks depth as it does not explain what the operation of addition entails. \n\nFinal Decision: [[B]]',
 'value': 'B',
 'score': 0}
```

### 기준 정의
##### criteria 기준은 다양한 기준 중 하나의 기준을 사용자 정의할 수 있다.
##### 아래는 사용자 정의 스타일을 기반으로 선호하는 작문 응답을 결정하는 예제이다.

```python
custom_criteria = {
    "simplicity": "Is the language straightforward and unpretentious?",
    "clarity": "Are the sentences clear and easy to understand?",
    "precision": "Is the writing precise, with no unnecessary words or details?",
    "truthfulness": "Does the writing feel honest and sincere?",
    "subtext": "Does the writing suggest deeper meanings or themes?",
}
evaluator = load_evaluator("pairwise_string", criteria=custom_criteria)

evaluator.evaluate_string_pairs(
    prediction="Every cheerful household shares a similar rhythm of joy; but sorrow, in each household, plays a unique, haunting melody.",
    prediction_b="Where one finds a symphony of joy, every domicile of happiness resounds in harmonious,"
    " identical notes; yet, every abode of despair conducts a dissonant orchestra, each"
    " playing an elegy of grief that is peculiar and profound to its own existence.",
    input="Write some prose about families.",
)

{'reasoning': 'Response A is simple, clear, and precise. It uses straightforward language to convey a deep and sincere message about families. The metaphor of joy and sorrow as music is effective and easy to understand.\n\nResponse B, on the other hand, is more complex and less clear. The language is more pretentious, with words like "domicile," "resounds," "abode," "dissonant," and "elegy." While it conveys a similar message to Response A, it does so in a more convoluted way. The precision is also lacking due to the use of unnecessary words and details.\n\nBoth responses suggest deeper meanings or themes about the shared joy and unique sorrow in families. However, Response A does so in a more effective and accessible way.\n\nTherefore, the better response is [[A]].',
 'value': 'A',
 'score': 1}
```


### 사용자 정의 LLM
##### 기본적으로 로더는 gpt-4 평가 체인에서 사용하지만, 이를 사용자 정의할 수 있다.

```python
from langchain_community.chat_models import ChatAnthropic

llm = ChatAnthropic(temperature=0)

evaluator = load_evaluator("labeled_pairwise_string", llm=llm)

evaluator.evaluate_string_pairs(
    prediction="there are three dogs",
    prediction_b="4",
    input="how many dogs are in the park?",
    reference="four",
)

{'reasoning': 'Here is my assessment:\n\nResponse B is more helpful, insightful, and accurate than Response A. Response B simply states "4", which directly answers the question by providing the exact number of dogs mentioned in the reference answer. In contrast, Response A states "there are three dogs", which is incorrect according to the reference answer. \n\nIn terms of helpfulness, Response B gives the precise number while Response A provides an inaccurate guess. For relevance, both refer to dogs in the park from the question. However, Response B is more correct and factual based on the reference answer. Response A shows some attempt at reasoning but is ultimately incorrect. Response B requires less depth of thought to simply state the factual number.\n\nIn summary, Response B is superior in terms of helpfulness, relevance, correctness, and depth. My final decision is: [[B]]\n',
 'value': 'B',
 'score': 0}
```

### 사용자 정의 평가 프롬프트
##### 자신만의 사용자 정의 평가 프롬프트를 사용하여 작업별 지침을 추가하거나 평가자에게 결과를 채점하도록 지시할 수 있다
##### 참고 : 고유한 형식으로 결과를 생성할려면 기본값 대신 사용자 정의 출력 파서(output_parser=your_parser())를 전달해야 할 수도 있다.

```python
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """Given the input context, which do you prefer: A or B?
Evaluate based on the following criteria:
{criteria}
Reason step by step and finally, respond with either [[A]] or [[B]] on its own line.

DATA
----
input: {input}
reference: {reference}
A: {prediction}
B: {prediction_b}
---
Reasoning:

"""
)
evaluator = load_evaluator("labeled_pairwise_string", prompt=prompt_template)
```

## 임베딩쌍 거리
##### 예측을 임베딩하고 두 임베딩 간의 벡터 거리를 계산한다.
##### 이를 수행하기 위해 pairwise_embedding_distance 평가기를 로드한다.

```python
from langchain.evaluation import load_evaluator

evaluator = load_evaluator("pairwise_embedding_distance")

evaluator.evaluate_string_pairs(
    prediction="Seattle is hot in June", prediction_b="Seattle is cool in June."
)

{'score': 0.0966466944859925}
```

### 거리 측정
##### 기본적으로 평가자는 코산 거리를 사용하지만, 다른 거리 측정법을 사용할 수 있다.

```python
from langchain.evaluation import EmbeddingDistance

list(EmbeddingDistance)

[<EmbeddingDistance.COSINE: 'cosine'>,
 <EmbeddingDistance.EUCLIDEAN: 'euclidean'>,
 <EmbeddingDistance.MANHATTAN: 'manhattan'>,
 <EmbeddingDistance.CHEBYSHEV: 'chebyshev'>,
 <EmbeddingDistance.HAMMING: 'hamming'>]

evaluator = load_evaluator(
    "pairwise_embedding_distance", distance_metric=EmbeddingDistance.EUCLIDEAN
)
```

### 사용할 임베딩 선택
##### 생성자는 OpenAI 임베딩을 기본적으로 사용하지만 원하는 대로 구성할 수 있다.

```python
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings()
hf_evaluator = load_evaluator("pairwise_embedding_distance", embeddings=embedding_model)

hf_evaluator.evaluate_string_pairs(
    prediction="Seattle is hot in June", prediction_b="Seattle is cool in June."
)
```

## 맞춤형 쌍 평가자
##### PairwiseStringEvaluator 클래스에서 상속하고 _evaluate_string_pairs 메서드 및 _aevaluate_string_pairs 메서드를 덮어써서 자신만의 문자열쌍 평가기를 만들 수 있다.

```python
from typing import Any, Optional

from langchain.evaluation import PairwiseStringEvaluator


class LengthComparisonPairwiseEvaluator(PairwiseStringEvaluator):
    """
    Custom evaluator to compare two strings.
    """

    def _evaluate_string_pairs(
        self,
        *,
        prediction: str,
        prediction_b: str,
        reference: Optional[str] = None,
        input: Optional[str] = None,
        **kwargs: Any,
    ) -> dict:
        score = int(len(prediction.split()) > len(prediction_b.split()))
        return {"score": score}
```

# Trajectory Evaluators(에이전트 궤적 평가자)
##### 궤적 평가자는 에이전트 평가에 대한 보다 전체적인 접근 방법을 제공한다.
##### 해당 평가자는 에이전트가 수행한 전체 작업 순서와 해당 응답을 평가하면 이를 "궤적"이라고 한다.
##### 궤적 평가자인 AgentTrajectoryEvaluator는 두 가지 주요 인터페이스를 구현한다.


*   evaluate_agent_trajectory : 에이전트의 궤적을 동기적으로 평가한다.
*   aevaluate_agent_trajectory : 비동기식 대응을 통해 평가를 병렬로 실행할 수 있다.

##### 두 방법 모두 세 가지 주요 매개변수를 허용한다.



*   input : 에이전트에 제공되는 초기 입력
*   prediction : 에이전트의 최종 에측 응답
*   agent_trajectory : 튜플 목록으로 제공되는 에이전트가 수행하는 중간 단계





## 사용자 정의 궤적 평가기
##### AgentTrajectoryEvaluator 클래스를 상속하고 _evaluate_agent_trajectory 및 _aevaluate_agent_trajectory 메서드를 덮어쓰면 사용자 정의 궤적 평가기를 만들 수 있다.

```python
from typing import Any, Optional, Sequence, Tuple

from langchain.chains import LLMChain
from langchain.evaluation import AgentTrajectoryEvaluator
from langchain_core.agents import AgentAction
from langchain_openai import ChatOpenAI


class StepNecessityEvaluator(AgentTrajectoryEvaluator):
    """Evaluate the perplexity of a predicted string."""

    def __init__(self) -> None:
        llm = ChatOpenAI(model="gpt-4", temperature=0.0)
        template = """Are any of the following steps unnecessary in answering {input}? Provide the verdict on a new line as a single "Y" for yes or "N" for no.

        DATA
        ------
        Steps: {trajectory}
        ------

        Verdict:"""
        self.chain = LLMChain.from_string(llm, template)

    def _evaluate_agent_trajectory(
        self,
        *,
        prediction: str,
        input: str,
        agent_trajectory: Sequence[Tuple[AgentAction, str]],
        reference: Optional[str] = None,
        **kwargs: Any,
    ) -> dict:
        vals = [
            f"{i}: Action=[{action.tool}] returned observation = [{observation}]"
            for i, (action, observation) in enumerate(agent_trajectory)
        ]
        trajectory = "\n".join(vals)
        response = self.chain.run(dict(trajectory=trajectory, input=input), **kwargs)
        decision = response.split("\n")[-1].strip()
        score = 1 if decision == "Y" else 0
        return {"score": score, "value": decision, "reasoning": response}
```

##### 위의 예에서 언어 모델 작업 중 하나라도 불필요하다고 예측하면 점수 1를 반환하고 모든 작업이 필요하다고 판단하면 점수 0을 반환한다.

```python
evaluator = StepNecessityEvaluator()

evaluator.evaluate_agent_trajectory(
    prediction="The answer is pi",
    input="What is today?",
    agent_trajectory=[
        (
            AgentAction(tool="ask", tool_input="What is today?", log=""),
            "tomorrow's yesterday",
        ),
        (
            AgentAction(tool="check_tv", tool_input="Watch tv for half hour", log=""),
            "bzzz",
        ),
    ],
)

{'score': 1, 'value': 'Y', 'reasoning': 'Y'}
```

## 에이전트 궤적
##### 에이전트는 수행할 수 있는 작업과 생성의 폭이 넓기 때문에 전체적으로 평가하기 힘들 수 있다. 그래서 에어전트 응답과 함께 수행되는 작업의 전체 궤적을 살펴보자
##### AgentTrajectoryEvaluator 클래스를 사용하여 평가자를 구현하자

```python
from langchain.evaluation import load_evaluator

evaluator = load_evaluator("trajectory")
```

##### 에이전트 궤적 평가자는 aevaluate_agent_trajectory 메서드를 사용하여 호출할 수 있다.

*   input : 에이전트에 대한 입력
*   prediction : 최종 예측 응답
*   Agent_trajectory : 에이전트 궤적을 형성하는 중간 단계

##### 다음 값을 사용하여 사전을 반환한다.


*   reasoning : LLM 문자열 (사고 추론)
*   score : 0에서 1, 1은 가장 효과적이고 0은 가장 효과적이지 않음

### 궤적 추적
##### 평가를 위해 에이전트의 궤적을 반환하는 가장 쉬운 방법은 return_intermediate_steps=True를 사용하여 에이전트를 초기화하는 것이다.

```python
import subprocess
from urllib.parse import urlparse

from langchain.agents import AgentType, initialize_agent
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from pydantic import HttpUrl


@tool
def ping(url: HttpUrl, return_error: bool) -> str:
    """Ping the fully specified url. Must include https:// in the url."""
    hostname = urlparse(str(url)).netloc
    completed_process = subprocess.run(
        ["ping", "-c", "1", hostname], capture_output=True, text=True
    )
    output = completed_process.stdout
    if return_error and completed_process.returncode != 0:
        return completed_process.stderr
    return output


@tool
def trace_route(url: HttpUrl, return_error: bool) -> str:
    """Trace the route to the specified url. Must include https:// in the url."""
    hostname = urlparse(str(url)).netloc
    completed_process = subprocess.run(
        ["traceroute", hostname], capture_output=True, text=True
    )
    output = completed_process.stdout
    if return_error and completed_process.returncode != 0:
        return completed_process.stderr
    return output


llm = ChatOpenAI(model="gpt-3.5-turbo-0613", temperature=0)
agent = initialize_agent(
    llm=llm,
    tools=[ping, trace_route],
    agent=AgentType.OPENAI_MULTI_FUNCTIONS,
    return_intermediate_steps=True,  # IMPORTANT!
)

result = agent("What's the latency like for https://langchain.com?")
```

### 궤적 평가
##### 입력, 궤적을 evaulate_agent_trajectory 메서드에 전달한다.

```python
evaluation_result = evaluator.evaluate_agent_trajectory(
    prediction=result["output"],
    input=result["input"],
    agent_trajectory=result["intermediate_steps"],
)

{'score': 1.0,
 'reasoning': "i. The final answer is helpful. It directly answers the user's question about the latency for the website https://langchain.com.\n\nii. The AI language model uses a logical sequence of tools to answer the question. It uses the 'ping' tool to measure the latency of the website, which is the correct tool for this task.\n\niii. The AI language model uses the tool in a helpful way. It inputs the URL into the 'ping' tool and correctly interprets the output to provide the latency in milliseconds.\n\niv. The AI language model does not use too many steps to answer the question. It only uses one step, which is appropriate for this type of question.\n\nv. The appropriate tool is used to answer the question. The 'ping' tool is the correct tool to measure website latency.\n\nGiven these considerations, the AI language model's performance is excellent. It uses the correct tool, interprets the output correctly, and provides a helpful and direct answer to the user's question."}
```

### 평가 LLM
##### 기본적으로 gpt-4를 사용하지만 임의적으로 변경할 수 있다.

```python
from langchain_community.chat_models import ChatAnthropic

eval_llm = ChatAnthropic(temperature=0)
evaluator = load_evaluator("trajectory", llm=eval_llm)
```

### 유용한 도구
##### 기본적으로 평가자는 에이전트가 호출하도록 허용된 도구를 고려하지 않아, agent_tools 인수를 통해 평가자에게 제공할 수 있다.

```python
from langchain.evaluation import load_evaluator

evaluator = load_evaluator("trajectory", agent_tools=[ping, trace_route])

evaluation_result = evaluator.evaluate_agent_trajectory(
    prediction=result["output"],
    input=result["input"],
    agent_trajectory=result["intermediate_steps"],
)

{'score': 1.0,
 'reasoning': "i. The final answer is helpful. It directly answers the user's question about the latency for the specified website.\n\nii. The AI language model uses a logical sequence of tools to answer the question. In this case, only one tool was needed to answer the question, and the model chose the correct one.\n\niii. The AI language model uses the tool in a helpful way. The 'ping' tool was used to determine the latency of the website, which was the information the user was seeking.\n\niv. The AI language model does not use too many steps to answer the question. Only one step was needed and used.\n\nv. The appropriate tool was used to answer the question. The 'ping' tool is designed to measure latency, which was the information the user was seeking.\n\nGiven these considerations, the AI language model's performance in answering this question is excellent."}
```